In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["GROQ_API_KEY"]= os.getenv("GROQ_API_KEY")

In [2]:
from langchain.chat_models import init_chat_model


model = init_chat_model("groq:llama-3.1-8b-instant")

e:\ONEXT\AI Research\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 8192, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x00000247A7BD4F70>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000247A7BD4E80>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
from pydantic import BaseModel,Field

class Movie(BaseModel):
    title :str = Field(description="The title of the movie")
    year :int = Field(description="This year the movie was released")
    director :str = Field(description="The director of the movie")
    rating :float = Field(description="The moive rating out of 10")

In [5]:
model_with_structured = model.with_structured_output(Movie)

In [6]:
model_with_structured

RunnableBinding(bound=ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 8192, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x00000247A7BD4F70>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000247A7BD4E80>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'Movie', 'description': '', 'parameters': {'properties': {'title': {'description': 'The title of the movie', 'type': 'string'}, 'year': {'description': 'This year the movie was released', 'type': 'integer'}, 'director': {'description': 'The director of the movie', 'type': 'string'}, 'rating': {'description': 'The moive rating out of 10', 'type': 'number'}}, 'required': ['title', 'yea

In [7]:
model.invoke("Provide details about the movie Inception write by Vietnamese")

AIMessage(content='Inception là một bộ phim kinh dị tâm lý được đạo diễn bởi Christopher Nolan và phát hành vào năm 2010. Bộ phim được viết bởi Christopher Nolan và có sự tham gia của các diễn viên nổi tiếng như Leonardo DiCaprio, Joseph Gordon-Levitt, Ellen Page, Tom Hardy, Ken Watanabe, Dileep Rao, Cillian Murphy, Marion Cotillard, và Tom Berenger.\n\nCốt truyện của Inception xoay quanh nhân vật Cobb (đóng bởi Leonardo DiCaprio), một kẻ trộm ý tưởng có khả năng vào giấc mơ. Cobb được một doanh nhân giàu có tên là Saito (đóng bởi Ken Watanabe) mời tham gia vào một dự án gọi là "inception" - việc làm cho ai đó nghĩ ra một ý tưởng mà không biết đó chính là ý tưởng của mình. Đổi lại, Saito sẽ cho phép Cobb trở về nước Mỹ với gia đình của mình.\n\nĐể đạt được mục tiêu này, Cobb và nhóm của mình bao gồm Arthur (đóng bởi Joseph Gordon-Levitt), Ariadne (đóng bởi Ellen Page), Eames (đóng bởi Tom Hardy), Saito, và một nhân vật bí ẩn tên là Robert Fischer (đóng bởi Cillian Murphy). Họ phải vào 

In [8]:
response = model_with_structured.invoke("Provide details about the movie Inception")
response

Movie(title='Inception', year=2010, director='Christopher Nolan', rating=8.8)

In [9]:
from pydantic import BaseModel,Field

class Movie(BaseModel):
    title :str = Field(...,description="The title of the movie")
    year :int = Field(...,description="This year the movie was released")
    director :str = Field(...,description="The director of the movie")
    rating :float = Field(...,description="The moive rating out of 10")

model_with_structured = model.with_structured_output(Movie, include_raw=True)

response = model_with_structured.invoke("Provide details about the movie Inception")
response


BadRequestError: Error code: 400 - {'error': {'message': "Failed to call a function. Please adjust your prompt. See 'failed_generation' for more details.", 'type': 'invalid_request_error', 'code': 'tool_use_failed', 'failed_generation': '<function=Movie> {"title": "Inception", "year": 2010, "director": "Christopher Nolan", "rating": 8.5}'}}

In [ ]:
# class Actor(BaseModel):
#     name:str
#     role:str

# class MovieDetails(BaseModel):
#     title:str
#     year:int
#     cast:list[Actor]
#     genres:list[str]
#     budget: float |None=Field(None, description="Budget in millions USD")

# model_with_structured = model.with_structured_output(MovieDetails)

# response = model_with_structured.invoke("What is the budget of the movie Inception?")
# response

### TyprDict

In [10]:
from typing_extensions import TypedDict, Annotated

class MovieDict(TypedDict):
    title: Annotated[str, "Title of the movie"]
    director: Annotated[str, "Director of the movie"]
    rating: Annotated[float, "Rating of the movie out of 10"]

model_with_typedict = model.with_structured_output(MovieDict)
response = model_with_typedict.invoke("Please provide me with the best movie of avengers.")
response

{'director': 'Anthony and Joe Russo',
 'rating': 8.7,
 'title': 'Avengers: Endgame'}

In [11]:
model.profile

{'max_input_tokens': 131072,
 'max_output_tokens': 8192,
 'image_inputs': False,
 'audio_inputs': False,
 'video_inputs': False,
 'image_outputs': False,
 'audio_outputs': False,
 'video_outputs': False,
 'reasoning_output': False,
 'tool_calling': True}

### Dataclass

In [12]:
from pydantic import BaseModel, Field
from langchain.agents import create_agent

class ContactInfo(BaseModel):
    name: str = Field(description="The name of the person")
    email: str = Field(description="The email address of the person")
    phone: str = Field(description="The phone number of the person")

agent = create_agent(
  model= "groq:llama-3.1-8b-instant",
  response_format=ContactInfo
)

result = agent.invoke({
  "messages": [{"role": "user", "content": "Extract contact info from: John Doe, Email: johndoe@example.com, Phone: 123-456-7890"}]
})

result["structured_response"]

ContactInfo(name='John Doe', email='johndoe@example.com', phone='123-456-7890')

In [13]:
#dataclass

from dataclasses import dataclass 
from langchain.agents import create_agent

@dataclass
class ContactInfo:
  name:str
  email:str
  phone:str


agent = create_agent(
  model= "groq:llama-3.1-8b-instant",
  response_format=ContactInfo
)

result["structured_response"]

ContactInfo(name='John Doe', email='johndoe@example.com', phone='123-456-7890')